In [1]:
import torch
import torch.nn as nn
from depth_anything_v2.dpt import DepthAnythingV2
from torch.utils.data import DataLoader, Dataset
import os
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torch.optim import AdamW
import torch.nn.functional as F
from torchvision import transforms

xFormers not available
xFormers not available


In [2]:
# Charger le modèle Depth Anything

model = DepthAnythingV2()

In [81]:
np.load("dataset/depth/" + os.listdir("dataset/depth/")[0]).shape

(1200, 1944, 3)

In [82]:
np.array(Image.open("dataset/images/" + os.listdir("dataset/images/")[0])).shape

(1200, 1944, 3)

In [3]:
# Intégration de LoRA

class LoRA(nn.Module):
    def __init__(self, layer, rank=4):
        super(LoRA, self).__init__()
        self.rank = rank
        self.layer = layer

        # Matrices de faible rang (U et V)
        self.U = nn.Parameter(torch.randn(layer.in_features, rank) * 0.01)
        self.V = nn.Parameter(torch.randn(rank, layer.out_features) * 0.01)

    def forward(self, x):
        # Adaptation avec les matrices de faible rang
        return self.layer(x) + torch.matmul(torch.matmul(x, self.U), self.V)

# Appliquer LoRA aux couches linéaires du modèle Depth Anything
layers_to_modify = []

for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        layers_to_modify.append((name, module))

for name, module in layers_to_modify:
    lora_layer = LoRA(module)
    setattr(model, name, lora_layer)

In [4]:
# Gel des paramètres non LoRA

for name, param in model.named_parameters():
    if 'U' not in name and 'V' not in name:  # Geler les autres paramètres
        param.requires_grad = False

In [59]:
# Création du dataset

def preprocess_data(image_path, xyz_path, target_size=(224, 224)):
    # Charger l'image RGB
    image = Image.open(image_path).convert("RGB")
    image = image.resize(target_size, Image.Resampling.LANCZOS)  # Redimensionner
    image = np.array(image) / 255.0  # Normaliser entre 0 et 1
    
    # Charger le nuage de points XYZ
    xyz = np.load(xyz_path)
    
    # Redimensionner le nuage de points pour correspondre à la taille cible
    xyz_resized = np.zeros((*target_size, 3), dtype=np.float32)
    for i in range(3):  # Canaux X, Y, Z
        xyz_resized[:, :, i] = np.array(Image.fromarray(xyz[:, :, i]).resize(target_size, Image.Resampling.LANCZOS))
    
    # Convertir en tenseurs PyTorch
    image_tensor = torch.from_numpy(image.transpose(2, 0, 1)).float()  # (H, W, C) -> (C, H, W)
    xyz_tensor = torch.from_numpy(xyz_resized)
    
    return image_tensor, xyz_tensor

# Dataset personnalisé
class DepthDataset(Dataset):
    def __init__(self, image_dir, image_files, point_cloud_dir, point_cloud_files):
        self.image_dir = image_dir
        self.image_files = image_files
        self.point_cloud_dir = point_cloud_dir
        self.point_cloud_files = point_cloud_files
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_files[idx])
        xyz_path = os.path.join(self.point_cloud_dir, self.point_cloud_files[idx])

        image, xyz = preprocess_data(image_path, xyz_path)
        
        return image, torch.tensor(xyz, dtype=torch.float32)

In [60]:
# Chargement des données

# Paramètres
IMAGES_DIR = "dataset/images"
DEPTH_DIR = "dataset/depth"
TRAIN_RATIO = 0.8

# Liste des fichiers
images_files = sorted(os.listdir(IMAGES_DIR))
depth_files = sorted(os.listdir(DEPTH_DIR))

# Vérification de correspondance
assert len(images_files) == len(depth_files), "Les dossiers images et depth doivent contenir le même nombre de fichiers."
assert all(img.split('.')[0][:-6] == depth.split('.')[0][:-9] for img, depth in zip(images_files, depth_files)), \
    "Les noms des fichiers images et depth doivent correspondre."

# Division des données
train_indices, val_indices = train_test_split(range(len(images_files)), train_size=TRAIN_RATIO, random_state=42)

# Création des listes d'entraînement et de validation
train_images = [images_files[idx] for idx in train_indices]
val_images = [images_files[idx] for idx in val_indices]
train_depth = [depth_files[idx] for idx in train_indices]
val_depth = [depth_files[idx] for idx in val_indices]

# Créations des datasets
train_dataset = DepthDataset(IMAGES_DIR, train_images, DEPTH_DIR, train_depth)
val_dataset = DepthDataset(IMAGES_DIR, val_images, DEPTH_DIR, val_depth)

# Division en mini-lots
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [61]:
train_dataset[0][0].shape, train_dataset[0][1].shape

C:\Users\estelle\AppData\Local\Temp\ipykernel_18056\1587601536.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return image, torch.tensor(xyz, dtype=torch.float32)


(torch.Size([3, 224, 224]), torch.Size([224, 224, 3]))

In [62]:
# Optimiser uniquement les paramètres des couches LoRA
params_to_update = []
for name, param in model.named_parameters():
    if 'U' in name or 'V' in name:  # Ne mettre à jour que les matrices U et V de LoRA
        if param.requires_grad:
            params_to_update.append(param)

optimizer = AdamW(params_to_update, lr=1e-4)

# Fonction de perte (par exemple, l'erreur de profondeur)
def depth_loss(pred, target):
    return F.mse_loss(pred, target)

num_epochs = 1
# Entraînement
model.train()
for epoch in range(num_epochs):
    for batch in train_loader:
        images, depth_maps = batch  # images et leurs cartes de profondeur cibles

        optimizer.zero_grad()
        outputs = model(images)  # Prédire la carte de profondeur
        loss = depth_loss(outputs, depth_maps)  # Calculer la perte
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

C:\Users\estelle\AppData\Local\Temp\ipykernel_18056\1587601536.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return image, torch.tensor(xyz, dtype=torch.float32)
C:\Users\estelle\AppData\Local\Temp\ipykernel_18056\3798512537.py:12: UserWarning: Using a target size (torch.Size([16, 224, 224, 3])) that is different to the input size (torch.Size([16, 224, 224])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(pred, target)


RuntimeError: The size of tensor a (224) must match the size of tensor b (3) at non-singleton dimension 3

In [63]:
print(f"Shape of outputs: {outputs.shape}")
print(f"Shape of depth_maps: {depth_maps.shape}")


Shape of outputs: torch.Size([16, 224, 224])
Shape of depth_maps: torch.Size([16, 224, 224, 3])


In [ ]:
# Evaluation des performances

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model.eval()
predictions = []
targets = []

# Exemple d'évaluation avec un DataLoader de test
for batch in test_loader:
    images, depth_maps = batch
    with torch.no_grad():
        outputs = model(images)

    predictions.append(outputs)
    targets.append(depth_maps)

# Calculer les métriques
predictions = torch.cat(predictions, dim=0).cpu().numpy()
targets = torch.cat(targets, dim=0).cpu().numpy()

# Conversion en valeurs discrètes pour le calcul des métriques (si nécessaire)
predictions = (predictions > 0.5).astype(int)
targets = (targets > 0.5).astype(int)

accuracy = accuracy_score(targets, predictions)
precision = precision_score(targets, predictions)
recall = recall_score(targets, predictions)
f1 = f1_score(targets, predictions)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

In [ ]:
# Sauvegarde des poids du modèle

torch.save(model.state_dict(), "depth_anything_lora_finetuned.pth")